In [105]:
import torch
import torchvision

In [126]:
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from torchvision.datasets import STL10
from torch.utils.data import DataLoader, Dataset

In [24]:
sys.path.insert(0, '/home/brendalf/Documents/projects/gan-attack/src')

In [113]:
def get_transform():
    transform = Compose([
        Resize((32,32)),
        ToTensor()     
    ])
    
    return transform

In [114]:
transform = get_transform()

In [115]:
trainset = STL10(
    root='../data', split='train', download=True, transform=transform
)

Files already downloaded and verified


In [119]:
import os
import torchvision

## Generating dataset with 100 images per class

In [124]:
new_folder = '../data/stl10'

In [125]:
if not os.path.exists(new_folder):
    os.mkdir(new_folder)

In [128]:
labels = {l:0 for l in np.arange(0, 10)}
num = 100

In [131]:
for image, label in trainset:
    label_folder = os.path.join(new_folder, str(label))

    if not os.path.exists(label_folder):
        os.mkdir(label_folder)
    
    if labels[label] == 100:
        continue

    torchvision.utils.save_image(image, fp=os.path.join(label_folder, f'{labels[label]}.png'))
    labels[label] += 1

## Stealing labels from target

In [132]:
from models.target.cifar10 import Cifar10Custom

In [133]:
model = torch.load('../models/target/cifar10.custom.pth')

In [136]:
from torchvision.datasets import ImageFolder

In [139]:
dataset = ImageFolder(root=new_folder, transform=transform)

In [148]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

In [149]:
from tqdm import tqdm
  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

real_labels = []
pred_labels = []

print('Generating labels from target...')
with torch.no_grad():
    model.eval()
    
    with open(output_labels, 'w') as output_fd:
        for images, labels in tqdm(dataloader):
            real_labels.extend(labels.numpy())
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            pred_labels.extend(predicted.cpu().numpy())
            
            output_fd.writelines(
                ['{},{}\n'.format(img_fn, label) for img_fn, label 
                    in zip(filenames, predicted)
                ]
            )

100%|██████████| 32/32 [00:00&lt;00:00, 157.72it/s]


In [49]:
import numpy as np

In [150]:
(unique, counts) = np.unique(real_labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[  0, 100],
       [  1, 100],
       [  2, 100],
       [  3, 100],
       [  4, 100],
       [  5, 100],
       [  6, 100],
       [  7, 100],
       [  8, 100],
       [  9, 100]])

In [151]:
(unique, counts) = np.unique(pred_labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[  0, 128],
       [  1,  21],
       [  2,  67],
       [  3, 295],
       [  4,  93],
       [  5,  86],
       [  6,   9],
       [  7,  30],
       [  8, 245],
       [  9,  26]])